目标是自定义recall precision fscore并加入到tensorboard中

## 检查GPU

In [7]:
!nvidia-smi

Thu Dec 21 19:26:30 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 0000:02:00.0     Off |                  N/A |
|  0%   34C    P0    55W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     Off  | 0000:03:00.0     Off |                  N/A |
| 23%   21C    P8     8W / 250W |  10797MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

## 导入常用库

In [8]:
import matplotlib.pyplot as plt
import openslide
from openslide import OpenSlide
import cv2
import numpy as np
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 导入fashionmnist并划分数据

In [3]:
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_train = pd.read_csv('/home/hjr/data/fashionmnist/fashion-mnist_train.csv')
data_test = pd.read_csv('/home/hjr/data/fashionmnist/fashion-mnist_test.csv')

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))

#Here we split validation data to optimiza classifier during training
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

#Test data
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_test /= 255
X_val /= 255

print('train-set')
X_train.shape
y_train.shape
print('validation-set')
X_val.shape
y_val.shape
print('test-set')
X_test.shape
y_test.shape

Using TensorFlow backend.


train-set


(48000, 28, 28, 1)

(48000, 10)

validation-set


(12000, 28, 28, 1)

(12000, 10)

test-set


(10000, 28, 28, 1)

(10000, 10)

## 构建cnn模型

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

batch_size = 256
num_classes = 10
epochs = 50

#input image dimensions
img_rows, img_cols = 28, 28

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))




##  定义指标和优化器

In [19]:
import keras.backend as K

def precision(y_true, y_pred):
    # 平均precison
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    # 本来就是正确的和所有预测的正确的，那么就计算T-T和所有的直
    # [0 1 0 0 0]*[0.1 0.6. 0.1 0.1 0.1]=[0 0.6 0 0 0 ]再round,相当于阈值0.5
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # 每一个y_pred都有一个1
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # 平均recall
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [21]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy', recall, precision, f1])

## 定义batch_size和epochs

In [9]:
BATCH_SIZE = 250
epochs = 50

## 创建子文件夹用于记录

In [10]:
!mkdir -p fashionmnist-cnn-12-21/tensorboard_log
!mkdir -p fashionmnist-cnn-12-21/model-checkpoint
!mkdir -p fashionmnist-cnn-12-21/csv-log

## 定义callback

In [17]:
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger, EarlyStopping


# because tensorboard needs directory, it can create, but csv_logger and model both are files, 
# so if you want to lay them on subdirectory you have to create subdirectory by yourself




# 用tensorboard显示，每次重新训练的话放到不同的文件里或者清空
# TensorBoard callback saves loss/accuracy data for visualization with tensorboard
log_dir_path = "fashionmnist-cnn-12-21/tensorboard_log/first"
tf_board = TensorBoard(log_dir=log_dir_path, batch_size=BATCH_SIZE, write_graph=True)

# 只保存最好的，以val_acc作为指标，保存整个模型,覆盖式写法
model_save_path = "fashionmnist-cnn-12-21/model-checkpoint/{epoch:02d}-{val_acc:.3f}.hdf5"
model_checkpt = ModelCheckpoint(model_save_path, monitor='val_acc', 
                                verbose=0, save_best_only=True, mode='max')

# 保存每次的loss/accuracy，覆盖式写法
# CSVLogger callbacks writes loss/accuracy output per epoch to file for later post-processing and visualization
csv_logger_path = 'fashionmnist-cnn-12-21/csv-log/train.log'
csv_logger = CSVLogger(csv_logger_path)

callbacks_list = [tf_board, model_checkpt, csv_logger]


## 训练

In [23]:
# fit时如果关掉了浏览器的tab，依然会训练，可以通过tensorboard和model-checkpoint或者csv-log看出
history = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=5,
          verbose=1,
          callbacks = callbacks_list,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 44us/step - loss: 0.1643 - acc: 0.9368 - recall: 0.9311 - precision: 0.9435 - f1: 0.9372 - val_loss: 0.2217 - val_acc: 0.9222 - val_recall: 0.9168 - val_precision: 0.9305 - val_f1: 0.9235
Epoch 2/5
48000/48000 [==============================] - 2s 44us/step - loss: 0.1695 - acc: 0.9359 - recall: 0.9302 - precision: 0.9430 - f1: 0.9366 - val_loss: 0.2274 - val_acc: 0.9206 - val_recall: 0.9145 - val_precision: 0.9283 - val_f1: 0.9214
Epoch 3/5
48000/48000 [==============================] - 2s 44us/step - loss: 0.1663 - acc: 0.9367 - recall: 0.9300 - precision: 0.9436 - f1: 0.9367 - val_loss: 0.2152 - val_acc: 0.9242 - val_recall: 0.9186 - val_precision: 0.9316 - val_f1: 0.9250
Epoch 4/5
48000/48000 [==============================] - 2s 44us/step - loss: 0.1625 - acc: 0.9385 - recall: 0.9320 - precision: 0.9447 - f1: 0.9383 - val_loss: 0.2140 - val_acc: 0.9241 - val_recall: 0.9175